
# PCR Single-Model Diagnostic (30m Train, 6m Test)

Goal:
- Diagnose why `baseline + PCR` degraded in recent workflow.
- Train/test a single ticker model with **30 months train** and **6 months test**.
- Compare against residual baseline (`0.0`) and inspect model stability.


In [ ]:

import numpy as np
import pandas as pd
import pickle
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)


In [ ]:

DATA_DIR = Path('data') if Path('data/raw/adr_info.csv').exists() else Path('../data')
MODEL_DIR = DATA_DIR / 'processed' / 'models' / 'with_us_stocks' / 'pcr'
FEAT_DIR = DATA_DIR / 'processed' / 'models' / 'with_us_stocks' / 'features_extended'
SIGNAL_DIR = DATA_DIR / 'processed' / 'index_russell_pcr_signal'
BASELINE_DIR = DATA_DIR / 'processed' / 'futures_only_signal'

assert MODEL_DIR.exists()
assert FEAT_DIR.exists()


In [ ]:

def compute_ic(pred, y):
    a = np.asarray(pred, dtype=float)
    b = np.asarray(y, dtype=float)
    m = np.isfinite(a) & np.isfinite(b)
    if m.sum() < 5:
        return np.nan
    a = a[m]
    b = b[m]
    if np.std(a) == 0 or np.std(b) == 0:
        return np.nan
    return float(np.corrcoef(a, b)[0, 1])


def fit_pcr_with_val(X_train, y_train, X_val, y_val, comp_grid):
    scaler = StandardScaler().fit(X_train)
    Xt = scaler.transform(X_train)
    Xv = scaler.transform(X_val)

    max_comp = int(min(Xt.shape[0], Xt.shape[1]))
    grid = sorted(set([int(k) for k in comp_grid if 1 <= int(k) <= max_comp]))
    if not grid:
        grid = [min(40, max_comp)]

    pca_all = PCA(n_components=max(grid), svd_solver='randomized', random_state=42)
    Zt_all = pca_all.fit_transform(Xt)
    Zv_all = pca_all.transform(Xv)
    zt_norm2 = np.sum(Zt_all * Zt_all, axis=0)
    zt_norm2 = np.where(zt_norm2 <= 1e-12, 1e-12, zt_norm2)

    best = (-np.inf, None)
    y_train = np.asarray(y_train, dtype=float)
    for k in grid:
        beta = (Zt_all[:, :k].T @ y_train) / zt_norm2[:k]
        pred = Zv_all[:, :k] @ beta
        ic = compute_ic(pred, y_val)
        if np.isfinite(ic) and ic > best[0]:
            best = (ic, int(k))

    # Refit on train+val with chosen k
    Xfull = np.vstack([X_train, X_val])
    yfull = np.concatenate([y_train, y_val])
    scaler = StandardScaler().fit(Xfull)
    Xf = scaler.transform(Xfull)
    k = best[1] if best[1] is not None else min(40, Xf.shape[1], Xf.shape[0])
    k = int(max(1, min(k, Xf.shape[0], Xf.shape[1])))

    pca = PCA(n_components=k, svd_solver='randomized', random_state=42)
    Zf = pca.fit_transform(Xf)
    zf_norm2 = np.sum(Zf * Zf, axis=0)
    zf_norm2 = np.where(zf_norm2 <= 1e-12, 1e-12, zf_norm2)
    beta = (Zf.T @ np.asarray(yfull, dtype=float)) / zf_norm2

    w_scaled = pca.components_.T @ beta.astype(np.float32)
    scale = np.where(scaler.scale_.astype(np.float32) <= 1e-6, 1e-6, scaler.scale_.astype(np.float32))
    mean = scaler.mean_.astype(np.float32)
    w_raw = w_scaled / scale
    c_raw = -float(np.dot(mean / scale, w_scaled))

    return {
        'val_ic': best[0],
        'n_components': k,
        'w_raw': w_raw,
        'c_raw': c_raw,
    }


## 1) Artifact stability check (recent months)

In [ ]:

def read_model_row(ticker, pkl_path):
    md = pickle.load(open(pkl_path, 'rb'))
    w = np.asarray(md.get('w_raw', []), dtype=float)
    return {
        'ticker': ticker,
        'model_file': pkl_path.name,
        'model_date': md.get('model_date'),
        'n_components': md.get('n_components'),
        'val_ic': md.get('val_ic'),
        'test_ic': md.get('test_ic'),
        'w_max_abs': float(np.nanmax(np.abs(w))) if w.size else np.nan,
        'c_raw': md.get('c_raw'),
        'n_features': len(md.get('feature_names', [])),
    }

rows = []
for ticker in ['ASML', 'NVO', 'AEG', 'BABA']:
    tdir = MODEL_DIR / ticker
    for p in sorted(tdir.glob('2025_*.pkl')) + sorted(tdir.glob('2026_*.pkl')):
        rows.append(read_model_row(ticker, p))

artifact_df = pd.DataFrame(rows)
artifact_df['model_date'] = pd.to_datetime(artifact_df['model_date'])
artifact_df = artifact_df.sort_values(['ticker', 'model_date'])
artifact_df.tail(20)


In [ ]:

# Highlight unstable models by coefficient scale
unstable = artifact_df[artifact_df['w_max_abs'] > 1e6].copy()
print('Unstable rows (w_max_abs > 1e6):', len(unstable))
unstable[['ticker', 'model_file', 'model_date', 'n_components', 'val_ic', 'test_ic', 'w_max_abs']].head(30)


## 2) Single-model train/test (30m train, 6m test)

In [ ]:

# Diagnostic split around weak period
TRAIN_START = pd.Timestamp('2023-03-01')
TRAIN_END = pd.Timestamp('2025-08-31')
TEST_START = pd.Timestamp('2025-09-01')
TEST_END = pd.Timestamp('2026-02-28')

# Internal validation from the last 6 months of train period
VAL_START = pd.Timestamp('2025-03-01')
VAL_END = pd.Timestamp('2025-08-31')
CORE_TRAIN_START = TRAIN_START
CORE_TRAIN_END = pd.Timestamp('2025-02-28')

COMP_GRIDS = {
    'cap40': [1,2,5,10,20,40],
    'cap80': [1,2,5,10,20,40,80],
    'cap120': [1,2,5,10,20,40,80,120],
    'cap200': [1,2,5,10,20,40,80,120,200],
    'aggressive': [1,2,5,10,20,40,80,120,200,400,800],
}

TICKERS = ['ASML', 'NVO']

results = []
for ticker in TICKERS:
    f = FEAT_DIR / f'{ticker}.parquet'
    df = pd.read_parquet(f)
    dt = pd.to_datetime(df['date']) if 'date' in df.columns else pd.to_datetime(df.index)
    feature_cols = [c for c in df.columns if c.startswith('russell_')]

    X = df[feature_cols].to_numpy(dtype=float)
    y = df['ordinary_residual'].to_numpy(dtype=float)

    m_core = (dt >= CORE_TRAIN_START) & (dt <= CORE_TRAIN_END)
    m_val = (dt >= VAL_START) & (dt <= VAL_END)
    m_test = (dt >= TEST_START) & (dt <= TEST_END)

    Xtr, ytr = X[m_core], y[m_core]
    Xva, yva = X[m_val], y[m_val]
    Xte, yte = X[m_test], y[m_test]

    baseline_test_ic = compute_ic(np.zeros_like(yte), yte)

    for label, grid in COMP_GRIDS.items():
        fit = fit_pcr_with_val(Xtr, ytr, Xva, yva, grid)
        pred_test = Xte @ fit['w_raw'] + fit['c_raw']
        test_ic = compute_ic(pred_test, yte)

        results.append({
            'ticker': ticker,
            'config': label,
            'n_components': fit['n_components'],
            'val_ic': fit['val_ic'],
            'test_ic': test_ic,
            'baseline_test_ic': baseline_test_ic,
            'improvement_vs_baseline': test_ic - baseline_test_ic,
            'w_max_abs': float(np.nanmax(np.abs(fit['w_raw']))),
            'pred_test_std': float(np.nanstd(pred_test)),
        })

res_df = pd.DataFrame(results).sort_values(['ticker', 'improvement_vs_baseline'], ascending=[True, False])
res_df


In [ ]:

for t in TICKERS:
    d = res_df[res_df['ticker'] == t].sort_values('improvement_vs_baseline', ascending=False)
    print('
Ticker:', t)
    display(d[['config','n_components','val_ic','test_ic','baseline_test_ic','improvement_vs_baseline','w_max_abs','pred_test_std']])


## 3) Inference signal sanity (current pipeline outputs)

In [ ]:

sig_rows = []
for ticker in TICKERS:
    p = SIGNAL_DIR / f'ticker={ticker}' / 'data.parquet'
    if not p.exists():
        continue
    s = pd.read_parquet(p)
    if s.empty:
        continue
    s.index = pd.to_datetime(s.index)
    m = s.index >= pd.Timestamp('2025-09-01', tz='America/New_York')
    x = s.loc[m, 'signal'].astype(float)
    if x.empty:
        continue
    sig_rows.append({
        'ticker': ticker,
        'n': len(x),
        'abs_q50': float(np.nanpercentile(np.abs(x), 50)),
        'abs_q95': float(np.nanpercentile(np.abs(x), 95)),
        'abs_q99': float(np.nanpercentile(np.abs(x), 99)),
        'abs_q999': float(np.nanpercentile(np.abs(x), 99.9)),
        'min': float(np.nanmin(x)),
        'max': float(np.nanmax(x)),
    })

pd.DataFrame(sig_rows)



## Interpretation guide

- If `w_max_abs` is huge and `pred_test_std` is huge under aggressive component settings, that indicates numerical instability.
- If capped-component configs keep `w_max_abs` and `pred_test_std` in normal ranges while preserving or improving test IC, that suggests a clear mitigation path:
  1. cap PCR components,
  2. floor tiny PCA/scale denominators,
  3. optionally add signal clipping / robust training.
